In [1]:
import pymongo
import numpy as np
import pandas as pd
import datetime

In [2]:
def usingPymongo(year,month,day):
    client = pymongo.MongoClient('127.0.0.1')
    db = client['trafficn']
    colName = 'DY50_XQ_SSSJ_LS'
    dgxqCol = db[colName]

    dtStart = datetime.datetime(year, month, day, 0, 0, 0)
    #[start,end)
    dtEnd = datetime.datetime(year, month, day+1, 0, 0, 0)
    # QSSJ have been converted to datetime , not string in this collection
    cursor = dgxqCol.find({'QSSJ':{'$gte':dtStart, '$lt':dtEnd}},{'_id':0,'SBBH':1,'FX':1,'ZHLL':1,'QSSJ':1})\
        .sort('QSSJ', pymongo.ASCENDING)
    #print(cursor.count())
    df = pd.DataFrame(list(cursor))
    return df

In [3]:
%timeit usingPymongo(2015,7,1)

184126
184126
184126
184126
1 loops, best of 3: 2.56 s per loop


In [8]:
df =usingPymongo(2015,7,1)
print(df.head())

184126
   FX       QSSJ  SBBH  ZHLL
0   1 2015-07-01    26     1
1   2 2015-07-01    26     1
2   2 2015-07-01     2     1
3   1 2015-07-01     2     3
4   1 2015-07-01    22     9


In [5]:
def dgxq_all():
    client = pymongo.MongoClient('127.0.0.1')
    db = client['trafficn']
    colName = 'DGXQ'
    taxiCol = db[colName]
    cursor = taxiCol.find({},{'_id':0,'SBDM':1,'SBMC':1})\
            .sort('SBDM', pymongo.ASCENDING)
    df = pd.DataFrame(list(cursor))
    return df

In [7]:
df2 = dgxq_all()
print(df2.head())

   SBDM  SBMC
0     1  施家北路
1     2  施家南路
2     3  木业大道
3     4  谈公北路
4     5  谈公北路


In [9]:
df['SBBH'].head()

0    26
1    26
2     2
3     2
4    22
Name: SBBH, dtype: int64

In [11]:
merged = pd.merge(df,df2,left_on='SBBH',right_on='SBDM')
print(merged.head())

   FX                QSSJ  SBBH  ZHLL  SBDM  SBMC
0   1 2015-07-01 00:00:00    26     1    26  人民大道
1   2 2015-07-01 00:00:00    26     1    26  人民大道
2   1 2015-07-01 00:00:30    26     2    26  人民大道
3   2 2015-07-01 00:00:30    26     2    26  人民大道
4   1 2015-07-01 00:01:00    26     3    26  人民大道


In [14]:
merged.dtypes

FX               int64
QSSJ    datetime64[ns]
SBBH             int64
ZHLL             int64
SBDM             int64
SBMC            object
dtype: object

In [15]:
merged['ZHLL'].max()

24

In [16]:
merged['ZHLL'].min()

0

In [20]:
delta=datetime.timedelta(minutes=30)
print delta

0:30:00


In [23]:
dt = datetime.datetime(2015,7,1,0,0,0)
print dt

2015-07-01 00:00:00


In [27]:
start = dt + delta
print start

2015-07-01 00:30:00


In [28]:
len(merged[merged['QSSJ']<start])

3812

In [29]:
end = start + delta
print end

2015-07-01 01:00:00


In [33]:
filtered = merged[(merged['QSSJ']>=start) & (merged['QSSJ']<end)]
print len(filtered)
filtered.head()

3840


,FX,QSSJ,SBBH,ZHLL,SBDM,SBMC
120,1,2015-07-01 00:30:00,26,2,26,人民大道
121,2,2015-07-01 00:30:00,26,2,26,人民大道
122,1,2015-07-01 00:30:30,26,1,26,人民大道
123,2,2015-07-01 00:30:30,26,4,26,人民大道
124,1,2015-07-01 00:31:00,26,2,26,人民大道


In [41]:
print merged.index
merged.set_index(pd.DatetimeIndex(merged['QSSJ']),inplace=True)
print merged.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9, 
            ...
            184116, 184117, 184118, 184119, 184120, 184121, 184122, 184123,
            184124, 184125],
           dtype='int64', length=184126)
DatetimeIndex(['2015-07-01 00:00:00', '2015-07-01 00:00:00',
               '2015-07-01 00:00:30', '2015-07-01 00:00:30',
               '2015-07-01 00:01:00', '2015-07-01 00:01:00',
               '2015-07-01 00:01:30', '2015-07-01 00:01:30',
               '2015-07-01 00:02:00', '2015-07-01 00:02:00', 
               ...
               '2015-07-01 23:57:30', '2015-07-01 23:57:30',
               '2015-07-01 23:58:00', '2015-07-01 23:58:00',
               '2015-07-01 23:58:30', '2015-07-01 23:58:30',
               '2015-07-01 23:59:00', '2015-07-01 23:59:00',
               '2015-07-01 23:59:30', '2015-07-01 23:59:30'],
              dtype='datetime64[ns]', length=184126, freq=None, tz=None)


In [44]:
filtered = merged.between_time(start_time=start,end_time=end,include_end=False)
print len(filtered)
filtered.head()

3840


,FX,QSSJ,SBBH,ZHLL,SBDM,SBMC
2015-07-01 00:30:00,1,2015-07-01 00:30:00,26,2,26,人民大道
2015-07-01 00:30:00,2,2015-07-01 00:30:00,26,2,26,人民大道
2015-07-01 00:30:30,1,2015-07-01 00:30:30,26,1,26,人民大道
2015-07-01 00:30:30,2,2015-07-01 00:30:30,26,4,26,人民大道
2015-07-01 00:31:00,1,2015-07-01 00:31:00,26,2,26,人民大道


In [53]:
result = filtered.groupby([filtered['SBMC'],filtered['SBDM'],filtered['FX']]).mean()
type(result)
result

SBBH      ZHLL
SBMC SBDM FX                
亭桥南路 19   1     19  3.566667
          2     19  3.550000
     20   1     20  4.083333
          2     20  3.216667
     21   1     21  3.283333
          2     21  3.666667
     22   1     22  3.116667
          2     22  3.233333
人民大道 24   1     24  0.383333
          2     24  0.616667
     25   1     25  1.300000
          2     25  1.283333
     26   1     26  1.383333
          2     26  2.050000
体育北路 8    1      8  0.000000
          2      8  0.000000
     9    1      9  7.416667
          2      9  3.150000
体育南路 10   1     10  5.833333
          2     10  5.866667
     11   1     11  5.466667
          2     11  5.600000
     12   1     12  3.833333
          2     12  1.533333
兴善大道 13   1     13  4.600000
          2     13  4.083333
施家南路 2    1      2  3.283333
          2      2  3.383333
晋阳东路 16   1     16  3.050000
          2     16  2.950000
...            ...       ...
晋阳西路 14   1     14  0.650000
          2     14  0.000000
     15   1     15  2.266667
          2     15  2.500000
环北东路 27   1     27  4.666667
          2     27  4.500000
     28   1     28  3.983333
          2     28  5.433333
     31   1     31  3.833333
          2     31  3.333333
环北西路 29   1     29  3.566667
          2     29  4.600000
     30   1     30  3.283333
          2     30  4.316667
解放东路 32   1     32  3.416667
          2     32  3.050000
     33   1     33  3.266667
          2     33  2.750000
     34   1     34  4.050000
          2     34  3.933333
解放西路 35   1     35  3.116667
          2     35  3.433333
     36   1     36  3.166667
          2     36  3.750000
谈公北路 4    1      4  7.266667
          2      4  6.983333
     5    1      5  6.033333
          2      5  5.500000
谈公南路 7    1      7  6.466667
          2      7  5.183333

[64 rows x 2 columns]

In [60]:
grouped = filtered.groupby([filtered['SBMC'],filtered['SBDM'],filtered['FX']])

In [64]:
for (name,group) in grouped:
#     print name, group
    print name, group['ZHLL'].mean()

(u'\u4ead\u6865\u5357\u8def', 19, 1) 3.56666666667
(u'\u4ead\u6865\u5357\u8def', 19, 2) 3.55
(u'\u4ead\u6865\u5357\u8def', 20, 1) 4.08333333333
(u'\u4ead\u6865\u5357\u8def', 20, 2) 3.21666666667
(u'\u4ead\u6865\u5357\u8def', 21, 1) 3.28333333333
(u'\u4ead\u6865\u5357\u8def', 21, 2) 3.66666666667
(u'\u4ead\u6865\u5357\u8def', 22, 1) 3.11666666667
(u'\u4ead\u6865\u5357\u8def', 22, 2) 3.23333333333
(u'\u4eba\u6c11\u5927\u9053', 24, 1) 0.383333333333
(u'\u4eba\u6c11\u5927\u9053', 24, 2) 0.616666666667
(u'\u4eba\u6c11\u5927\u9053', 25, 1) 1.3
(u'\u4eba\u6c11\u5927\u9053', 25, 2) 1.28333333333
(u'\u4eba\u6c11\u5927\u9053', 26, 1) 1.38333333333
(u'\u4eba\u6c11\u5927\u9053', 26, 2) 2.05
(u'\u4f53\u80b2\u5317\u8def', 8, 1) 0.0
(u'\u4f53\u80b2\u5317\u8def', 8, 2) 0.0
(u'\u4f53\u80b2\u5317\u8def', 9, 1) 7.41666666667
(u'\u4f53\u80b2\u5317\u8def', 9, 2) 3.15
(u'\u4f53\u80b2\u5357\u8def', 10, 1) 5.83333333333
(u'\u4f53\u80b2\u5357\u8def', 10, 2) 5.86666666667
(u'\u4f53\u80b2\u5357\u8def', 11, 1) 5.